<a href="https://colab.research.google.com/github/Strix9289/Introduction-of-Deep-Learning/blob/master/Activation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 活性化関数：Activate
* sigmoid: $f(x)=\dfrac{1}{1+e^{-x}}$
* ReLU: $f(x)=\max(0,x)$
* tanh: $f(x)=\tanh(x)=\dfrac{e^x-e^{-x}}{e^x+e^{-x}}$
* softmax: $f(x)=\dfrac{\exp(x_d)}{\sum_{d'} \exp(x_{d'})} \quad (x\in\mathbb{R}^D,\ d=1,2,\ldots,D)$ 予想確立を出す

# 活性化関数

**勾配消失問題**：ニューラルネットワークは**誤差逆電波法**によって、勾配が計算されている

つまり、**(ある相の勾配) = (1層前の勾配) x (2層前の勾配) x ... x (出力層の勾配)**と積の形で求めるため、

途中のから勾配の大きさが小さくなると入力層付近の勾配はどんどん0に近づく。

- Tanhの場合、入力値が0から離れると勾配が0に近くなるため、層が深くなると最適化が進みづらい
- ReLUの場合、常に勾配が1。前の層に勾配が伝わりやすく演算もシンプルで端末実装も容易。
- LeakyReLU: 0よりも小さな値にも勾配があり、勾配もパラメタとして学習できるPReLUもある。実装が重い
- ELU: 0よりも小さな値にも滑らかに変化する勾配がある。Softplus、SELUもこれに近い関数。実装が重い
‐ CReLU: CNNのフィルタには単純に符合を反転させたもののペアが存在する場合がある。これをCReLUを実装することで符号反転バージョンを作り、パラメタ数を減らすことができる。
- Swish: 強化学習を用いて発見された活性化関数。
